<a href="https://colab.research.google.com/github/DmitryKutsev/congrats_generator/blob/feature%233/my_gpt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb#scrollTo=aZ-4Kav28cH0




# **IMPORT**

In [1]:
import pandas as pd

In [2]:
!pip install transformers -U

     |████████████████████████████████| 3.4 MB 7.3 MB/s 
     |████████████████████████████████| 61 kB 437 kB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 3.3 MB 27.7 MB/s 
     |████████████████████████████████| 895 kB 57.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 6.8 MB/s 


In [4]:
!pip install tqdm boto3 requests regex sacremoses

     |████████████████████████████████| 131 kB 8.7 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 8.5 MB 8.2 MB/s 
     |████████████████████████████████| 138 kB 46.7 MB/s 
     |████████████████████████████████| 127 kB 45.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [6]:
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2')   
#  # Download model and configuration from huggingface.co and cache.
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', './test/saved_model/')  
# # E.g. model was saved using `save_pretrained('./test/saved_model/')`
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2', output_attentions=True)  
# # Update configuration during loading
# assert model.config.output_attentions == True
# # Loading from a TF checkpoint file instead of a PyTorch model (slower)
# config = AutoConfig.from_pretrained('bert-base-uncased')
# # AutoConfig.from_pretrained('bert-base-uncased')
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 
# #                        './tf_model/gpt_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [7]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 93155, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 93155 (delta 9), reused 44 (delta 7), pack-reused 93095
Receiving objects: 100% (93155/93155), 77.76 MiB | 13.04 MiB/s, done.
Resolving deltas: 100% (67382/67382), done.


In [8]:
!pip install tensorboard
# !pip install wandb; wandb login

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [10]:
!pip install awscli
!aws s3 sync --no-sign-request s3://models.dobro.ai/gpt2/ru/unfreeze_all gpt2

     |████████████████████████████████| 3.8 MB 8.6 MB/s 
     |████████████████████████████████| 547 kB 60.6 MB/s 
     |████████████████████████████████| 636 kB 68.6 MB/s 
  Attempting uninstall: rsa
    Found existing installation: rsa 4.8
    Uninstalling rsa-4.8:
      Successfully uninstalled rsa-4.8
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/config.json to gpt2/m_checkpoint-3364613/config.json
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/step.txt to gpt2/m_checkpoint-3364613/step.txt
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/step.txt to gpt2/s_checkpoint-1900000/step.txt
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/training_args.bin to gpt2/m_checkpoint-3364613/training_args.bin
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/training_args.bin to gpt2/s_checkpoint-1900000/training_args.bin
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/config.json to gpt2/s_checkpoint-1900000/config.json
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/s_checkpoint-1900000/encoder.model to gpt2/s_checkpoint-1900000/encoder.model
download: s3://models.dobro.ai/gpt2/ru/unfreeze_all/m_checkpoint-3364613/encoder.model to gpt2/m_check

# **PREPARE DATA**

In [11]:
!wget https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/feature%233/crawler/curr_congrats.txt

--2021-12-21 18:59:03--  https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/feature%233/crawler/curr_congrats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1471875 (1.4M) [text/plain]
Saving to: ‘curr_congrats.txt’

curr_congrats.txt   100%[===================>]   1.40M  --.-KB/s    in 0.04s   

2021-12-21 18:59:04 (34.1 MB/s) - ‘curr_congrats.txt’ saved [1471875/1471875]



In [12]:
with open('curr_congrats.txt') as handler:
  full_text = handler.read()
full_text[:3000]

'\nTITLE\nЭдварду Радзинскому, писателю, драматургу, сценаристу\nCONTENT\n Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанных\nдраматургов, мастеров современного театрального искусства. И\xa0конечно, Вас\nхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьё\nтворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодрости\nдуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      \nEND\n\nTITLE\nКоллективу радиостанции «Орфей»\nCONTENT\n Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение луч

In [13]:
full_text.split('END')[:3][0].replace('\n', '').split('CONTENT')

['TITLEЭдварду Радзинскому, писателю, драматургу, сценаристу',
 ' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ']

In [14]:
titles_list = []
congrats_list = []

for letter in full_text.split('END'):
  letter = letter.replace('\n', '').replace('TITLE', '')
  # print(letter)
  # print(letter.split('CONTENT'))
  # print(letter.split('CONTENT'))
  # break
  if len(letter.split('CONTENT')) > 1:
    titles_list.append(letter.split('CONTENT')[0])
    congrats_list.append(letter.split('CONTENT')[1])

print(titles_list[:3])
print(congrats_list[:3])
print(len(titles_list))
print(len(congrats_list))

['Эдварду Радзинскому, писателю, драматургу, сценаристу', 'Коллективу радиостанции «Орфей»', 'Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»']
[' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ', ' Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вкла

In [15]:
data = {'Title': titles_list,
        'Content': congrats_list}

my_df = pd.DataFrame(data)
my_df

,Title,Content
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...
...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...


In [16]:
! pip install datasets transformers

     |████████████████████████████████| 306 kB 8.6 MB/s 
     |████████████████████████████████| 132 kB 54.9 MB/s 
     |████████████████████████████████| 243 kB 60.3 MB/s 
     |████████████████████████████████| 1.1 MB 62.1 MB/s 
     |████████████████████████████████| 160 kB 22.4 MB/s 
     |████████████████████████████████| 271 kB 44.0 MB/s 
     |████████████████████████████████| 192 kB 14.2 MB/s 


In [17]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 683 (delta 110), reused 141 (delta 83), pack-reused 505
Receiving objects: 100% (683/683), 413.81 KiB | 2.13 MiB/s, done.
Resolving deltas: 100% (410/410), done.


In [18]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

In [19]:
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Content'].tolist()[i] for i in range(len(my_df))]
my_df

,Title,Content,Sum
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...,"Эдварду Радзинскому, писателю, драматургу, сце..."
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...,Коллективу радиостанции «Орфей» Дорогие друзья...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск..."
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...,"Ирине Андреевой и Ивану Штылю, победителям чем..."
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...,Работникам и ветеранам оборонно-промышленного ...
...,...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...,Жителям Республики Мордовия Дорогие друзья!Поз...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...,Коллективу Института физических проблем им.П.Л...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...,Жителям Красноярского края Уважаемые друзья!По...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...,В.А.ЛОГИНОВУ Уважаемый Владимир Александрович!...


In [20]:
train_test_ratio = 0.9
df_train, df_test = train_test_split(my_df, train_size = train_test_ratio, random_state = 1)
# train_valid_ratio = 7/9
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [21]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Sum'].tolist()
    # c = 0
    # if c < 2:
    #   print(summaries)
    #   c += 1
    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        summary = re.sub(r"\n", "", summary)
        summary = re.sub(r"Статус материала Опубликован в разделе: Телеграммы", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"Текстовая версия:", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"([a-zA-Z]+)", "", summary)
        summary = re.sub(r"([0-9/*#@+]+)", "", summary)
        # bos_token = '<BOS>'
        # eos_token = '<EOS>'
        # data += bos_token + ' ' + summary + ' ' + eos_token + '\n'
        data += summary
    # c = 0
    # if c < 2:
    #   print(data)
    #   c += 1
    f.write(data)

In [22]:
!ls
!rm 'train.txt' 'test.txt'
!ls

curr_congrats.txt  gpt2  ru-gpts  sample_data  transformers
rm: cannot remove 'train.txt': No such file or directory
rm: cannot remove 'test.txt': No such file or directory
curr_congrats.txt  gpt2  ru-gpts  sample_data  transformers


In [23]:
build_dataset(df_train, 'train.txt')
build_dataset(df_test, 'test.txt')
# build_dataset(df_valid, 'valid.txt')

In [24]:


# tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
 
train_path = 'train.txt'
test_path = 'test.txt'

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [25]:
!rm ./gpt-letters

rm: cannot remove './gpt-letters': No such file or directory


In [26]:
# tokenizer.add_special_tokens({'bos_token': '<BOS>', 'eos_token': '<EOS>', 'mask_token': '<MASK>'})

In [27]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## **TRAIN MODEL**

In [28]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="./gpt-letters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    # num_train_epochs=3, # number of training epochs
    # per_device_train_batch_size=32, # batch size for training
    # per_device_eval_batch_size=64,  # batch size for evaluation
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=24,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=150, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [29]:
trainer.train()

***** Running training *****
  Num examples = 989
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 186


Step,Training Loss


Saving model checkpoint to ./gpt-letters/checkpoint-150
Configuration saved in ./gpt-letters/checkpoint-150/config.json
Model weights saved in ./gpt-letters/checkpoint-150/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=186, training_loss=2.654388919953377, metrics={'train_runtime': 5124.6923, 'train_samples_per_second': 0.579, 'train_steps_per_second': 0.036, 'total_flos': 193813364736000.0, 'train_loss': 2.654388919953377, 'epoch': 3.0})

In [30]:
trainer.save_model()

Saving model checkpoint to ./gpt-letters
Configuration saved in ./gpt-letters/config.json
Model weights saved in ./gpt-letters/pytorch_model.bin


In [31]:
# my_model = torch.load('./gpt-letters/pytorch_model.bin')
from transformers import pipeline

pu = pipeline('text-generation',model='./gpt-letters', tokenizer=tokenizer, max_length=500)

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt-letters",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_

In [32]:
pu("Александру Сергеевичу Пушкину")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александру Сергеевичу Пушкину, народному артисту России Уважаемый Александр Сергеевич! Примите мои поздравления с -летием и самые добрые пожелания. Желаю Вам здоровья, благополучия и всего самого доброго. Владимир Путин       .   Текстовая версияАлександру Сергеевичу Пушкину, народному артисту России Уважаемый Александр Сергеевич! Примите мои поздравления с -летием и самые добрые пожелания. Желаю Вам здоровья, благополучия и всего самого доброго. Владимир Путин       .   Текстовая версияАлександру Сергеевичу Пушкину, народному артисту России Уважаемый Александр Сергеевич! Примите мои поздравления с -летием и самые добрые пожелания. Желаю Вам здоровья, благополучия и всего самого доброго. Владимир Путин       .   Текстовая версияАлександру Сергеевичу Пушкину, народному артисту России Уважаемый Александр Сергеевич! Примите мои поздравления с -летием и самые добрые пожелания. Желаю Вам здоровья, благополучия и всего самого доброго. Владимир Путин       .   Текстовая версияАлександру Серг

In [33]:
pu("Кровавым убийцам детей")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Кровавым убийцам детей и молодёжи Уважаемые друзья!Поздравляю вас с -летием создания Российского военно-исторического общества.В его стенах вы впервые познакомитесь с историей создания и развития Российского военно-исторического общества, с его богатейшим историческим, культурным, научным, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просветительским, просве

In [34]:
pu("Уважаемые осквернители святынь и насильники")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Уважаемые осквернители святынь и насильники!Поздравляю вас с праздником – Днём рождения Государственного Эрмитажа.Вы по праву принадлежите к числу выдающихся мастеров отечественной культуры, которые внесли большой вклад в развитие отечественной культуры, внесли значимый вклад в развитие отечественной культуры.Уверен, что и впредь вы будете достойно нести вахту служения искусству, бережно хранить богатейшие коллекции, бережно передавать их из поколения в поколение.Желаю вам здоровья, благополучия и всего самого доброго. Владимир Путин       .   Текстовая версияКоллективу и ветеранам Государственного Эрмитажа Дорогие друзья!Поздравляю вас с Днём рождения Государственного Эрмитажа.Вы по праву принадлежите к числу выдающихся мастеров отечественной культуры, внесли большой вклад в развитие отечественной культуры, внесли значимый вклад в развитие отечественной культуры.Уверен, что и впредь вы будете достойно нести вахту служения искусству, бережно хранить богатейшие коллекции, бережно перед

In [35]:
pu("Хочу сказать этому Обэме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Хочу сказать этому Обэме, что вы по праву принадлежите к числу тех, кто в течение многих лет бережно хранит и приумножает традиции своего народа, бережно хранит и передаёт из поколения в поколение лучшие традиции и традиции своего народа.Желаю вам здоровья, благополучия и всего самого доброго. Владимир Путин       .   Текстовая версияКоллективу и ветеранам Государственного академического театра драмы имени А.С.Пушкина Дорогие друзья!Поздравляю вас с -летием Государственного академического театра драмы имени А.С.Пушкина.Вы по праву принадлежите к числу тех, кто в течение многих лет бережно хранит и приумножает традиции своего народа, бережно хранит и передаёт из поколения в поколение лучшие традиции и традиции своего народа, бережно хранит и передаёт из поколения в поколение лучшие традиции и традиции своего народа, бережно хранит и передаёт из поколения в поколение лучшие традиции и традиции своего народа, бережно хранит и передаёт из поколения в поколение лучшие традиции и традиции с

In [36]:
pu("Куцеву Диме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Куцеву Диме, победителю  Паралимпийских летних игр в Токио в соревнованиях по плаванию в дисциплине  метров Уважаемый Дмитрий Анатольевич!Поздравляю Вас с триумфом.Вы блестяще выступили на  Паралимпийских летних играх в Токио в соревнованиях по плаванию в дисциплине  метров.Вы блестяще выступили на  Паралимпийских летних играх в Токио в соревнованиях по плаванию в дисциплине  метров.Вы блестяще выступили на  Паралимпийских летних играх в Токио в соревнованиях по плаванию в дисциплине  метров.Вы блестяще выступили на  Паралимпийских летних играх в Токио в соревнованиях по плаванию в дисциплине  метров.Вы блестяще выступили на  Паралимпийских летних играх в Токио в соревнованиях по плаванию в дисциплине  метров.Вы блестяще выступили на  Паралимпийских летних играх в Токио в соревнованиях по плаванию в дисциплине  метров.Вы блестяще выступили на  Паралимпийских летних играх в Токио в соревнованиях по плаванию в дисциплине  метров.Вы блестяще выступили на  Паралимпийских летних играх в То

In [37]:
pu("Мычалкин")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Мычалкин, заслуженный мастер спорта СССР, заслуженный мастер спорта СССР Уважаемый Владимир Иванович! Примите мои поздравления с -летием и самые добрые пожелания. Желаю Вам здоровья, благополучия и всего самого доброго. Владимир Путин       .   Текстовая версияКоллективу и ветеранам Государственного академического Большого театра России Дорогие друзья!Поздравляю вас с -летием Государственного академического Большого театра России.Вы по праву принадлежите к числу выдающихся театральных деятелей, которые внесли весомый вклад в развитие отечественного театра, внесли значимый вклад в развитие отечественной культуры.Важно, что сегодня Государственный академический Большой театр России является одним из крупнейших театров России, одним из крупнейших театров России, одним из крупнейших театров России, одним из крупнейших театров России, одним из крупнейших театров России, одним из крупнейших театров России, одним из крупнейших театров России, одним из крупнейших театров России, одним из круп

In [38]:
from google.colab import files

In [39]:
import os
for file in os.listdir('./gpt-letters'):
  files.download(f'./gpt-letters/{file}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
import os
for file in os.listdir('./gpt-letters'):
  print(file)

pytorch_model.bin
config.json
checkpoint-150
training_args.bin
runs
